In [8]:
import torch
import keras
import resnet
import numpy as np
import torchvision

In [2]:
model, embedding = resnet.__dict__['resnet50'](zero_init_residual=True)
state_dict = torch.load('vicreg_pure/vicreg/exp/resnet50_100epochs_16_factor.pth', map_location="cpu")
if "model" in state_dict:
    state_dict = state_dict["model"]
    state_dict = {key.replace("module.backbone.", ""): value for (key, value) in state_dict.items()}
model.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [10]:
model = torchvision.models.resnet50(pretrained=True)

In [11]:
import onnx
import numpy

x = torch.randn(1, 3, 224, 224, requires_grad=True)
torch.onnx.export(model, x, "torchToOnnx.onnx", verbose=True, input_names = ['input'], output_names = ['output'])

graph(%input : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=1, device=cpu),
      %fc.weight : Float(1000, 2048, strides=[2048, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %497 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %498 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %500 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %501 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %503 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %504 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %506 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %507 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %509 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %510 : Float(256, strides=[1], requir

In [12]:
import onnx
from onnx2keras import onnx_to_keras
# Load ONNX model
onnx_model = onnx.load('torchToOnnx.onnx')
# Call the converter (input will be equal to the input_names parameter that you defined during exporting)
k_model = onnx_to_keras(onnx_model, ['input'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight fc.weight with shape (1000, 2048).
DEBUG:onnx2keras:Found weight fc.bias with shape (1000,).
DEBUG:onnx2keras:Found weight onnx::Conv_497 with shape (64, 3, 7, 7).
DEBUG:onnx2keras:Found weight onnx::Conv_498 with shape (64,).
DEBUG:onnx2keras:Found weight onnx::Conv_500 with shape (64, 64, 1, 1).
DEBUG:onnx2keras:Found weight onnx::Conv_501 with shape (64,).
DEBUG:onnx2keras:Found weight onnx::Conv_503 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight onnx::Conv_504 with shape (64,).
DEBUG:onnx2keras:Found weight onnx::Conv_506 with shape (256, 64, 1, 1).
DEBUG:onnx2keras:Found weight onnx::Conv_507 with shape (256,).
DEBUG:onnx2keras:Found weight onnx::Conv_509 with shape (2

DEBUG:onnx2keras:Check input 1 (name onnx::Conv_497).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name onnx::Conv_498).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 64, 112, 112), dtype=tf.float32, name=None), name='input.4/BiasAdd:0', description="created by layer 'input.4'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Relu
DEBUG:onnx2keras:node_name: onnx::MaxPool_323
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2kera

ValueError: 'onnx::MaxPool_323/' is not a valid root scope name. A root scope name has to match the following pattern: ^[A-Za-z0-9.][A-Za-z0-9_.\\/>-]*$

In [11]:
from pytorch2keras import pytorch_to_keras

def check_error(output, k_model, input_np, epsilon=1e-3):
    pytorch_output = output.data.numpy()
    keras_output = k_model.predict(input_np)

    error = np.max(pytorch_output - keras_output)
    print('Error:', error)

    assert error < epsilon
    return error


if __name__ == '__main__':
    max_error = 0
    for i in range(100):
        
        c
        
        model.eval()

        input_np = np.random.uniform(0, 1, (1, 3, 224, 224))
        input_var = torch.autograd.Variable(torch.FloatTensor(input_np))
        output = model(input_var)

        k_model = pytorch_to_keras(model, input_var, (3, 224, 224,), verbose=True)

        error = check_error(output, k_model, input_np)
        if max_error < error:
            max_error = error

    print('Max error: {0}'.format(max_error))

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']


graph(%input_0 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %495 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %496 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %498 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %499 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %501 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %502 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %504 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %505 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %507 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %508 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %510 : Float(64, 256, 1, 1, strides=[256, 1, 1, 1], requires_grad=0, device=cpu),
      %511 : Float(64, strides=[1], requires

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[(3, 224, 224)]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight onnx::Conv_495 with shape (64, 3, 7, 7).
DEBUG:onnx2keras:Found weight onnx::Conv_496 with shape (64,).
DEBUG:onnx2keras:Found weight onnx::Conv_498 with shape (64, 64, 1, 1).
DEBUG:onnx2keras:Found weight onnx::Conv_499 with shape (64,).
DEBUG:onnx2keras:Found weight onnx::Conv_501 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight onnx::Conv_502 with shape (64,).
DEBUG:onnx2keras:Found weight onnx::Conv_504 with shape (256, 64, 1, 1).
DEBUG:onnx2keras:Found weight onnx::Conv_505 with shape (256,).
DEBUG:onnx2keras:Found weight onnx::Conv_507 with shape (256, 64, 1, 1).
DEBUG:onnx2keras:Found weight onnx::Conv_508 with shape (256,).
DEBUG:onnx2keras:Found weight

DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: input.8
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [7, 7], 'pads': [2, 2, 2, 2], 'strides': [2, 2], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input).
DEBUG:onnx2keras:Check input 1 (name onnx::Conv_495).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name onnx::Conv_496).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:Output T

[0, 0, 1, 1, 0, 0, 1, 1]


ValueError: 'onnx::MaxPool_322/' is not a valid root scope name. A root scope name has to match the following pattern: ^[A-Za-z0-9.][A-Za-z0-9_.\\/>-]*$